## Quantize a Segmentation Model and Show Live Inference

In [1]:
"""
On Windows, try to find the directory that contains x64 cl.exe and add it to the PATH to enable PyTorch
to find the required C++ tools. This code assumes that Visual Studio is installed in the default
directory. If you have a different C++ compiler, please add the correct path to os.environ["PATH"]
directly. Note that the C++ Redistributable is not enough to run this notebook.

Adding the path to os.environ["LIB"] is not always required - it depends on the system's configuration
"""

from re import search
import sys

if sys.platform == "win32":
    import distutils.command.build_ext
    import os
    from pathlib import Path

    if sys.getwindowsversion().build >= 20000:
        search_path = "**/Hostx64/x64/cl.exe"    
    else:
        search_path = "**/Hostx86/x64/cl.exe"

    VS_INSTALL_DIR_2019 = r"C:/Program Files (x86)/Microsoft Visual Studio"
    VS_INSTALL_DIR_2022 = r"C:/Program Files/Microsoft Visual Studio"

    cl_paths_2019 = sorted(list(Path(VS_INSTALL_DIR_2019).glob(search_path)))
    cl_paths_2022 = sorted(list(Path(VS_INSTALL_DIR_2022).glob(search_path)))
    cl_paths = cl_paths_2019 + cl_paths_2022

    if len(cl_paths) == 0:
        raise ValueError(
            """
            Cannnot find Visual Studio. This notebook requires an x64 C++ compiler. If you installed a
            C++ compiler, please add the directory that contains cl.exe to `os.environ[PATH]`
            """
        )

    else:
        cl_path = cl_paths[-1]
        vs_dir = str(cl_path.parent)
        os.environ["PATH"] += f"{os.pathsep}{vs_dir}"
        d =  distutils.core.Distribution()
        b = distutils.command.build_ext.build_ext(d)
        b.finalize_options()
        os.environ["LIB"] = os.pathsep.join(b.library_dirs)
        print(f"Added {vs_dir} to PATH")

Added C:\Program Files\Microsoft Visual Studio\2022\Community\VC\Tools\MSVC\14.30.30705\bin\Hostx86\x64 to PATH


In [2]:
import logging
import os
import sys
import time
from tkinter.ttk import Notebook
import warnings
import zipfile
from pathlib import Path

warnings.filterwarnings("ignore", category=UserWarning)

import cv2
import matplotlib.pyplot as plt
import monai
import numpy as np
import torch
from monai.transforms import LoadImage
from nncf.common.utils.logger import set_log_level
from openvino.inference_engine import IECore
from openvino.runtime import Core
from torch.jit import TracerWarning
from torchmetrics import F1

set_log_level(logging.ERROR)

sys.path.append("../utils/")
from models.custom_segmentation import SegmentationModel
from notebook_utils import NotebookAlert, benchmark_model, download_file, show_live_inference

try:
    import subprocess
    from nncf import NNCFConfig
    from nncf.torch import create_compressed_model, register_default_init_args
except subprocess.CalledProcessError:
    message = "WARNING: Running this notebook requires an x64 C++ compiler"
    NotebookAlert(message=message, alert_class="warning")


No CUDA runtime is found, using CUDA_HOME='C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.6'


### Settings

In [ ]:
BASEDIR = Path("kits19_frames_1")
# Uncomment the line below to use the full dataset, as prepared in the data preparation notebook
# BASEDIR = Path("~/kits19/kits19_frames").expanduser()
MODEL_DIR = Path("model")
MODEL_DIR.mkdir(exist_ok=True)